# Khám Phá Dữ Liệu: Phân Tích Rời Bỏ Khách Hàng Thẻ Tín Dụng

## Mục tiêu
- Tải và kiểm tra bộ dữ liệu BankChurners sử dụng NumPy
- Hiểu cấu trúc dữ liệu và các đặc trưng
- Thực hiện phân tích khám phá dữ liệu
- Xử lý giá trị thiếu và tiền xử lý dữ liệu
- Chuẩn bị dữ liệu cho trực quan hóa và mô hình hóa

## Câu hỏi nghiên cứu:
Trước khi đi sâu vào dữ liệu, chúng ta đặt ra một số câu hỏi để định hướng phân tích:
1. **Chất lượng dữ liệu:** Dữ liệu có sạch không? Có giá trị thiếu hay ngoại lai nào đáng lo ngại không?
2. **Cấu trúc khách hàng:** Tỷ lệ khách hàng rời bỏ là bao nhiêu? Có sự mất cân bằng lớp nghiêm trọng không?
3. **Đặc điểm nhân khẩu học:** Khách hàng của ngân hàng chủ yếu thuộc nhóm tuổi, giới tính và trình độ học vấn nào?
4. **Mối quan hệ:** Có mối tương quan nào rõ ràng giữa các biến số (ví dụ: hạn mức tín dụng và thu nhập) ngay từ cái nhìn đầu tiên không?

In [1]:
# Import các thư viện cần thiết
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Đặt seed ngẫu nhiên để tái lập kết quả
np.random.seed(42)

## 1. Tải Bộ Dữ Liệu Sử Dụng NumPy

Chúng ta sẽ tải file CSV sử dụng hàm genfromtxt của NumPy để đọc dữ liệu vào mảng có cấu trúc.

In [2]:
# Tải dữ liệu sử dụng NumPy
data_path = '../data/raw/BankChurners.csv'

# Đầu tiên, đọc header
with open(data_path, 'r') as f:
    header = f.readline().strip().replace('"', '').split(',')

print("Tên các cột:")
for i, col in enumerate(header):
    print(f"{i}: {col}")

# Tải dữ liệu số và chuỗi
data_raw = np.genfromtxt(data_path, delimiter=',', skip_header=1, dtype=str, encoding='utf-8')

# Loại bỏ dấu ngoặc kép từ dữ liệu chuỗi (do file CSV có chứa dấu ngoặc kép)
data_raw = np.char.replace(data_raw, '"', '')

print(f"\nKích thước bộ dữ liệu: {data_raw.shape}")
print(f"Số lượng mẫu: {data_raw.shape[0]}")
print(f"Số lượng đặc trưng: {data_raw.shape[1]}")

Tên các cột:
0: CLIENTNUM
1: Attrition_Flag
2: Customer_Age
3: Gender
4: Dependent_count
5: Education_Level
6: Marital_Status
7: Income_Category
8: Card_Category
9: Months_on_book
10: Total_Relationship_Count
11: Months_Inactive_12_mon
12: Contacts_Count_12_mon
13: Credit_Limit
14: Total_Revolving_Bal
15: Avg_Open_To_Buy
16: Total_Amt_Chng_Q4_Q1
17: Total_Trans_Amt
18: Total_Trans_Ct
19: Total_Ct_Chng_Q4_Q1
20: Avg_Utilization_Ratio
21: Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1
22: Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2

Kích thước bộ dữ liệu: (10127, 23)
Số lượng mẫu: 10127
Số lượng đặc trưng: 23


## 2. Kiểm Tra Dữ Liệu

Hãy kiểm tra một vài hàng đầu tiên và hiểu kiểu dữ liệu của mỗi cột.

In [3]:
# Hiển thị 5 hàng đầu tiên
print("5 hàng đầu tiên của bộ dữ liệu:\n")
print("=" * 150)
for i, col in enumerate(header[:10]):  # Hiển thị 10 cột đầu tiên cho dễ đọc
    print(f"{col:30s}", end=" | ")
print("\n" + "=" * 150)

for row in data_raw[:5]:
    for val in row[:10]:
        print(f"{val:30s}", end=" | ")
    print()

# Xác định biến mục tiêu
print(f"\n\nBiến mục tiêu: {header[1]} (Attrition_Flag)")
print(f"Giá trị duy nhất trong biến mục tiêu: {np.unique(data_raw[:, 1])}")

5 hàng đầu tiên của bộ dữ liệu:

CLIENTNUM                      | Attrition_Flag                 | Customer_Age                   | Gender                         | Dependent_count                | Education_Level                | Marital_Status                 | Income_Category                | Card_Category                  | Months_on_book                 | 
768805383                      | Existing Customer              | 45                             | M                              | 3                              | High School                    | Married                        | $60K - $80K                    | Blue                           | 39                             | 
818770008                      | Existing Customer              | 49                             | F                              | 5                              | Graduate                       | Single                         | Less than $40K                 | Blue                           | 44      

In [4]:
# Phân tách đặc trưng theo loại
# Loại bỏ 2 cột cuối (Naive Bayes classifiers - không phải đặc trưng hữu ích)
header = header[:-2]
data_raw = data_raw[:, :-2]

# Định nghĩa các loại đặc trưng
categorical_features = [1, 3, 5, 6, 7, 8]  # Attrition_Flag, Gender, Education_Level, Marital_Status, Income_Category, Card_Category
numerical_features = [i for i in range(len(header)) if i not in categorical_features and i != 0]  # Loại trừ CLIENTNUM

print("Đặc trưng phân loại:")
for idx in categorical_features:
    print(f"  - {header[idx]}")

print(f"\nĐặc trưng số ({len(numerical_features)} đặc trưng):")
for idx in numerical_features[:10]:  # Hiển thị 10 đặc trưng đầu
    print(f"  - {header[idx]}")
print("  ...")

Đặc trưng phân loại:
  - Attrition_Flag
  - Gender
  - Education_Level
  - Marital_Status
  - Income_Category
  - Card_Category

Đặc trưng số (14 đặc trưng):
  - Customer_Age
  - Dependent_count
  - Months_on_book
  - Total_Relationship_Count
  - Months_Inactive_12_mon
  - Contacts_Count_12_mon
  - Credit_Limit
  - Total_Revolving_Bal
  - Avg_Open_To_Buy
  - Total_Amt_Chng_Q4_Q1
  ...


## 3. Kiểm Tra Giá Trị Thiếu

Sử dụng NumPy để phát hiện giá trị thiếu trong bộ dữ liệu.

In [5]:
# Kiểm tra giá trị thiếu (chuỗi rỗng, 'nan', 'NaN', None, v.v.)
def check_missing_values(data, columns):
    """Kiểm tra giá trị thiếu trong bộ dữ liệu sử dụng NumPy"""
    missing_count = np.zeros(data.shape[1])
    
    for col_idx in range(data.shape[1]):
        # Đếm chuỗi rỗng, 'nan', hoặc khoảng trắng
        col_data = data[:, col_idx]
        # Thêm 'Unknown' vào danh sách kiểm tra vì bộ dữ liệu này dùng 'Unknown' cho giá trị thiếu
        missing_mask = (col_data == '') | (col_data == 'nan') | (col_data == 'NaN') | (col_data == 'None') | (col_data == 'Unknown')
        missing_count[col_idx] = np.sum(missing_mask)
    
    return missing_count

missing_counts = check_missing_values(data_raw, header)

print("Tóm tắt giá trị thiếu:")
print("=" * 80)
print(f"{'Tên cột':<40} {'Số lượng thiếu':<15} {'Phần trăm':<15}")
print("=" * 80)

total_rows = data_raw.shape[0]
has_missing = False

for idx, col in enumerate(header):
    missing_cnt = int(missing_counts[idx])
    missing_pct = (missing_cnt / total_rows) * 100
    
    # In ra tất cả các cột, kể cả khi không có giá trị thiếu
    status = f"{missing_cnt:<15} {missing_pct:<15.2f}%" if missing_cnt > 0 else "0 (0.00%)"
    print(f"{col:<40} {status}")
    
    if missing_cnt > 0:
        has_missing = True

if not has_missing:
    print("\nKhông tìm thấy giá trị thiếu (bao gồm cả 'Unknown') trong bộ dữ liệu!")
else:
    print(f"\nLưu ý: Có {np.sum(missing_counts)} giá trị được đánh dấu là thiếu hoặc 'Unknown'.")
    
print("=" * 80)

Tóm tắt giá trị thiếu:
Tên cột                                  Số lượng thiếu  Phần trăm      
CLIENTNUM                                0 (0.00%)
Attrition_Flag                           0 (0.00%)
Customer_Age                             0 (0.00%)
Gender                                   0 (0.00%)
Dependent_count                          0 (0.00%)
Education_Level                          1519            15.00          %
Marital_Status                           749             7.40           %
Income_Category                          1112            10.98          %
Card_Category                            0 (0.00%)
Months_on_book                           0 (0.00%)
Total_Relationship_Count                 0 (0.00%)
Months_Inactive_12_mon                   0 (0.00%)
Contacts_Count_12_mon                    0 (0.00%)
Credit_Limit                             0 (0.00%)
Total_Revolving_Bal                      0 (0.00%)
Avg_Open_To_Buy                          0 (0.00%)
Total_Amt_Chng_Q4_Q

## 4. Tóm Tắt Thống Kê của Đặc Trưng Số

Tính toán thống kê mô tả sử dụng NumPy cho các đặc trưng số.

In [6]:
# Trích xuất đặc trưng số và chuyển sang float
numerical_data = np.zeros((data_raw.shape[0], len(numerical_features)))

for i, idx in enumerate(numerical_features):
    numerical_data[:, i] = data_raw[:, idx].astype(float)

# Tính toán thống kê sử dụng NumPy
def calculate_statistics(data):
    """Tính toán thống kê mô tả sử dụng NumPy"""
    stats = {
        'mean': np.mean(data, axis=0),
        'std': np.std(data, axis=0),
        'min': np.min(data, axis=0),
        'q1': np.percentile(data, 25, axis=0),
        'median': np.median(data, axis=0),
        'q3': np.percentile(data, 75, axis=0),
        'max': np.max(data, axis=0)
    }
    return stats

stats = calculate_statistics(numerical_data)

# Hiển thị thống kê cho 10 đặc trưng số đầu tiên
print("Tóm tắt thống kê (10 đặc trưng số đầu tiên):")
print("=" * 120)
print(f"{'Đặc trưng':<25} {'Trung bình':<12} {'Độ lệch chuẩn':<12} {'Min':<12} {'Trung vị':<12} {'Max':<12}")
print("=" * 120)

for i in range(min(10, len(numerical_features))):
    feature_name = header[numerical_features[i]]
    print(f"{feature_name:<25} {stats['mean'][i]:<12.2f} {stats['std'][i]:<12.2f} "
          f"{stats['min'][i]:<12.2f} {stats['median'][i]:<12.2f} {stats['max'][i]:<12.2f}")

print("=" * 120)

Tóm tắt thống kê (10 đặc trưng số đầu tiên):
Đặc trưng                 Trung bình   Độ lệch chuẩn Min          Trung vị     Max         
Customer_Age              46.33        8.02         26.00        46.00        73.00       
Dependent_count           2.35         1.30         0.00         2.00         5.00        
Months_on_book            35.93        7.99         13.00        36.00        56.00       
Total_Relationship_Count  3.81         1.55         1.00         4.00         6.00        
Months_Inactive_12_mon    2.34         1.01         0.00         2.00         6.00        
Contacts_Count_12_mon     2.46         1.11         0.00         2.00         6.00        
Credit_Limit              8631.95      9088.33      1438.30      4549.00      34516.00    
Total_Revolving_Bal       1162.81      814.95       0.00         1276.00      2517.00     
Avg_Open_To_Buy           7469.14      9090.24      3.00         3474.00      34516.00    
Total_Amt_Chng_Q4_Q1      0.76         0.22 

## 5. Phân Tích Đặc Trưng Phân Loại

Kiểm tra phân phối của các biến phân loại.

In [7]:
# Phân tích đặc trưng phân loại
def analyze_categorical(data, column_idx, column_name):
    """Phân tích đặc trưng phân loại sử dụng NumPy"""
    unique_values, counts = np.unique(data[:, column_idx], return_counts=True)
    percentages = (counts / len(data)) * 100
    
    print(f"\n{column_name}:")
    print("-" * 60)
    for val, cnt, pct in zip(unique_values, counts, percentages):
        print(f"  {val:<30} : {cnt:>6} ({pct:>6.2f}%)")
    
    return unique_values, counts

print("Phân tích đặc trưng phân loại:")
print("=" * 60)

# Phân tích biến mục tiêu
print("\n*** BIẾN MỤC TIÊU ***")
target_values, target_counts = analyze_categorical(data_raw, 1, header[1])

# Phân tích các đặc trưng phân loại khác
for idx in [3, 5, 6, 7, 8]:  # Gender, Education, Marital, Income, Card
    analyze_categorical(data_raw, idx, header[idx])

print("\n" + "=" * 60)

Phân tích đặc trưng phân loại:

*** BIẾN MỤC TIÊU ***

Attrition_Flag:
------------------------------------------------------------
  Attrited Customer              :   1627 ( 16.07%)
  Existing Customer              :   8500 ( 83.93%)

Gender:
------------------------------------------------------------
  F                              :   5358 ( 52.91%)
  M                              :   4769 ( 47.09%)

Education_Level:
------------------------------------------------------------
  College                        :   1013 ( 10.00%)
  Doctorate                      :    451 (  4.45%)
  Graduate                       :   3128 ( 30.89%)
  High School                    :   2013 ( 19.88%)
  Post-Graduate                  :    516 (  5.10%)
  Uneducated                     :   1487 ( 14.68%)
  Unknown                        :   1519 ( 15.00%)

Marital_Status:
------------------------------------------------------------
  Divorced                       :    748 (  7.39%)
  Married        

## 6. Tiền Xử Lý Dữ Liệu

### 6.1 Mã Hóa Biến Phân Loại

Chúng ta sẽ mã hóa các biến phân loại sử dụng NumPy (Label Encoding và One-Hot Encoding)

In [8]:
# Mã hóa biến mục tiêu (Attrition_Flag)
# "Attrited Customer" = 1, "Existing Customer" = 0
target = (data_raw[:, 1] == 'Attrited Customer').astype(int)

print(f"Mã hóa biến mục tiêu:")
print(f"  Attrited Customer (1): {np.sum(target)}")
print(f"  Existing Customer (0): {np.sum(target == 0)}")
print(f"  Tỷ lệ rời bỏ: {np.mean(target) * 100:.2f}%")

# Label Encoding cho đặc trưng phân loại nhị phân
def label_encode(column_data):
    """Label encoding đơn giản sử dụng NumPy"""
    unique_values = np.unique(column_data)
    encoding_map = {val: idx for idx, val in enumerate(unique_values)}
    encoded = np.array([encoding_map[val] for val in column_data])
    return encoded, encoding_map

# Mã hóa Gender (nhị phân)
gender_encoded, gender_map = label_encode(data_raw[:, 3])
print(f"\nMã hóa giới tính: {gender_map}")

# One-Hot Encoding cho đặc trưng phân loại đa lớp
def one_hot_encode(column_data):
    """One-hot encoding sử dụng NumPy"""
    unique_values = np.unique(column_data)
    n_values = len(unique_values)
    encoded = np.zeros((len(column_data), n_values))
    
    for idx, val in enumerate(unique_values):
        mask = (column_data == val)
        encoded[mask, idx] = 1
    
    return encoded, unique_values

# Mã hóa Education Level
education_encoded, education_classes = one_hot_encode(data_raw[:, 5])
print(f"\nTrình độ học vấn được mã hóa one-hot thành {education_encoded.shape[1]} lớp")
print(f"Các lớp: {education_classes}")

# Mã hóa Marital Status
marital_encoded, marital_classes = one_hot_encode(data_raw[:, 6])
print(f"\nTình trạng hôn nhân được mã hóa one-hot thành {marital_encoded.shape[1]} lớp")

# Mã hóa Income Category
income_encoded, income_classes = one_hot_encode(data_raw[:, 7])
print(f"\nHạng thu nhập được mã hóa one-hot thành {income_encoded.shape[1]} lớp")

# Mã hóa Card Category
card_encoded, card_classes = one_hot_encode(data_raw[:, 8])
print(f"\nLoại thẻ được mã hóa one-hot thành {card_encoded.shape[1]} lớp")

Mã hóa biến mục tiêu:
  Attrited Customer (1): 1627
  Existing Customer (0): 8500
  Tỷ lệ rời bỏ: 16.07%

Mã hóa giới tính: {np.str_('F'): 0, np.str_('M'): 1}

Trình độ học vấn được mã hóa one-hot thành 7 lớp
Các lớp: ['College' 'Doctorate' 'Graduate' 'High School' 'Post-Graduate'
 'Uneducated' 'Unknown']

Tình trạng hôn nhân được mã hóa one-hot thành 4 lớp

Hạng thu nhập được mã hóa one-hot thành 6 lớp

Loại thẻ được mã hóa one-hot thành 4 lớp


### 6.2 Phát Hiện và Xử Lý Giá Trị Ngoại Lai

Sử dụng phương pháp thống kê (phương pháp IQR) để phát hiện giá trị ngoại lai trong các đặc trưng số.

In [9]:
# Phát hiện giá trị ngoại lai sử dụng phương pháp IQR
def detect_outliers_iqr(data):
    """Phát hiện giá trị ngoại lai sử dụng phương pháp Interquartile Range (IQR)"""
    q1 = np.percentile(data, 25, axis=0)
    q3 = np.percentile(data, 75, axis=0)
    iqr = q3 - q1
    
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Tạo mask cho giá trị ngoại lai
    outliers_mask = (data < lower_bound) | (data > upper_bound)
    
    return outliers_mask, lower_bound, upper_bound

outliers_mask, lower_bounds, upper_bounds = detect_outliers_iqr(numerical_data)

print("Tóm tắt phát hiện giá trị ngoại lai:")
print("=" * 80)
print(f"{'Đặc trưng':<30} {'Số lượng ngoại lai':<20} {'Phần trăm':<15}")
print("=" * 80)

total_outliers = 0
for i in range(min(10, len(numerical_features))):
    outlier_count = np.sum(outliers_mask[:, i])
    outlier_pct = (outlier_count / len(numerical_data)) * 100
    feature_name = header[numerical_features[i]]
    print(f"{feature_name:<30} {outlier_count:<20} {outlier_pct:<15.2f}%")
    total_outliers += outlier_count

print("=" * 80)
print(f"\nTổng số trường hợp ngoại lai (qua 10 đặc trưng đầu): {total_outliers}")
print("\nLưu ý: Trong phân tích này, chúng ta sẽ giữ lại các giá trị ngoại lai vì chúng có thể")
print("đại diện cho hành vi khách hàng quan trọng (ví dụ: hạn mức tín dụng rất cao hoặc số tiền giao dịch).")

Tóm tắt phát hiện giá trị ngoại lai:
Đặc trưng                      Số lượng ngoại lai   Phần trăm      
Customer_Age                   2                    0.02           %
Dependent_count                0                    0.00           %
Months_on_book                 386                  3.81           %
Total_Relationship_Count       0                    0.00           %
Months_Inactive_12_mon         331                  3.27           %
Contacts_Count_12_mon          629                  6.21           %
Credit_Limit                   984                  9.72           %
Total_Revolving_Bal            0                    0.00           %
Avg_Open_To_Buy                963                  9.51           %
Total_Amt_Chng_Q4_Q1           396                  3.91           %

Tổng số trường hợp ngoại lai (qua 10 đặc trưng đầu): 3691

Lưu ý: Trong phân tích này, chúng ta sẽ giữ lại các giá trị ngoại lai vì chúng có thể
đại diện cho hành vi khách hàng quan trọng (ví dụ: hạn mức 

### 6.3 Tiêu Chuẩn Hóa Đặc Trưng (Standardization)

Áp dụng kỹ thuật Z-score Standardization để đưa các đặc trưng về cùng một phân phối chuẩn (mean=0, std=1). Điều này rất quan trọng đối với các thuật toán dựa trên khoảng cách và Gradient Descent (như Logistic Regression) để đảm bảo hội tụ nhanh và ổn định.

In [10]:
# Tiêu chuẩn hóa Z-score (mean=0, std=1)
def standardize(data):
    """Tiêu chuẩn hóa Z-score"""
    mean_vals = np.mean(data, axis=0)
    std_vals = np.std(data, axis=0)
    
    # Tránh chia cho 0
    std_vals[std_vals == 0] = 1
    
    standardized = (data - mean_vals) / std_vals
    return standardized, mean_vals, std_vals

# Áp dụng tiêu chuẩn hóa
standardized_data, mean_vals, std_vals = standardize(numerical_data)

print("Áp dụng tiêu chuẩn hóa Z-score:")
print("=" * 80)
print(f"{'Đặc trưng':<30} {'Trung bình':<15} {'Độ lệch chuẩn':<15}")
print("=" * 80)

for i in range(min(5, len(numerical_features))):
    feature_name = header[numerical_features[i]]
    print(f"{feature_name:<30} {standardized_data[:, i].mean():<15.6f} {standardized_data[:, i].std():<15.6f}")

print("=" * 80)

Áp dụng tiêu chuẩn hóa Z-score:
Đặc trưng                      Trung bình      Độ lệch chuẩn  
Customer_Age                   0.000000        1.000000       
Dependent_count                -0.000000       1.000000       
Months_on_book                 -0.000000       1.000000       
Total_Relationship_Count       0.000000        1.000000       
Months_Inactive_12_mon         0.000000        1.000000       


## 7. Feature Engineering: Tạo đặc trưng mới

Chúng ta sẽ tạo thêm các đặc trưng mới để nắm bắt sâu hơn hành vi khách hàng, dựa trên các giả thuyết về hành vi tiêu dùng và rủi ro.

### 1. Giá trị giao dịch trung bình (Avg_Transaction_Value)
- **Công thức:** `Total_Trans_Amt` / `Total_Trans_Ct`
- **Cơ sở lựa chọn:**
    - **Phân loại hành vi:** Giúp phân biệt rõ ràng giữa nhóm khách hàng "giao dịch thường xuyên, giá trị nhỏ" (như mua sắm tạp hóa) và nhóm "giao dịch ít, giá trị lớn" (mua sắm xa xỉ, du lịch).
    - **Dấu hiệu rời bỏ:** Khách hàng có ý định rời bỏ thường thay đổi hành vi chi tiêu. Một sự sụt giảm trong giá trị trung bình có thể chỉ ra rằng họ đã chuyển các giao dịch lớn sang thẻ của ngân hàng khác và chỉ dùng thẻ này cho các khoản nhỏ lẻ.

### 2. Tỷ lệ giao dịch trên hạn mức (Transaction_to_Limit_Ratio)
- **Công thức:** `Total_Trans_Amt` / `Credit_Limit`
- **Cơ sở lựa chọn:**
    - **Mức độ gắn kết (Engagement):** Chỉ số này đo lường mức độ "active" thực sự của khách hàng so với tiềm năng (hạn mức) được cấp.
    - **Dự báo rời bỏ:** Tỷ lệ này thấp (ví dụ: < 5%) cho thấy khách hàng đang "bỏ quên" thẻ hoặc chỉ giữ thẻ làm dự phòng (dormant users), đây là nhóm có nguy cơ rời bỏ cao nhất. Ngược lại, tỷ lệ sử dụng cao chứng tỏ sự phụ thuộc vào thẻ.

In [11]:
# 1. Giá trị giao dịch trung bình (Avg_Transaction_Value)
total_trans_amt_idx = header.index('Total_Trans_Amt')
total_trans_ct_idx = header.index('Total_Trans_Ct')

trans_amt = data_raw[:, total_trans_amt_idx].astype(float)
trans_ct = data_raw[:, total_trans_ct_idx].astype(float)

# Tránh chia cho 0
avg_trans_value = np.divide(trans_amt, trans_ct, out=np.zeros_like(trans_amt), where=trans_ct!=0)

# 2. Tỷ lệ giao dịch trên hạn mức (Transaction_to_Limit_Ratio)
credit_limit_idx = header.index('Credit_Limit')
credit_limit = data_raw[:, credit_limit_idx].astype(float)

# Tránh chia cho 0
trans_to_limit_ratio = np.divide(trans_amt, credit_limit, out=np.zeros_like(trans_amt), where=credit_limit!=0)

print("Thống kê các đặc trưng mới:")
print("=" * 60)
print(f"{'Đặc trưng':<30} {'Mean':<10} {'Min':<10} {'Max':<10}")
print("-" * 60)
print(f"{'Avg_Transaction_Value':<30} {np.mean(avg_trans_value):<10.2f} {np.min(avg_trans_value):<10.2f} {np.max(avg_trans_value):<10.2f}")
print(f"{'Transaction_to_Limit_Ratio':<30} {np.mean(trans_to_limit_ratio):<10.4f} {np.min(trans_to_limit_ratio):<10.4f} {np.max(trans_to_limit_ratio):<10.4f}")
print("=" * 60)

# Kiểm tra tương quan với đặc trưng gốc
print("\nTương quan với đặc trưng gốc (Total_Trans_Amt):")
corr_avg = np.corrcoef(avg_trans_value, trans_amt)[0, 1]
corr_ratio = np.corrcoef(trans_to_limit_ratio, trans_amt)[0, 1]
print(f"  - Avg_Transaction_Value vs Total_Trans_Amt: {corr_avg:.4f}")
print(f"  - Transaction_to_Limit_Ratio vs Total_Trans_Amt: {corr_ratio:.4f}")

Thống kê các đặc trưng mới:
Đặc trưng                      Mean       Min        Max       
------------------------------------------------------------
Avg_Transaction_Value          62.61      19.14      190.19    
Transaction_to_Limit_Ratio     1.0419     0.0181     5.0802    

Tương quan với đặc trưng gốc (Total_Trans_Amt):
  - Avg_Transaction_Value vs Total_Trans_Amt: 0.9121
  - Transaction_to_Limit_Ratio vs Total_Trans_Amt: 0.4186


## 8. Kiểm định giả thiết thống kê (Hypothesis Testing)

Sử dụng kiểm định T-test (Welch's t-test) để xác định xem có sự khác biệt có ý nghĩa thống kê về giá trị trung bình của các đặc trưng giữa nhóm khách hàng rời bỏ (Churn) và khách hàng hiện tại (Existing) hay không.

- **Giả thiết H0**: Không có sự khác biệt về giá trị trung bình giữa hai nhóm.
- **Giả thiết H1**: Có sự khác biệt về giá trị trung bình giữa hai nhóm.
- **Mức ý nghĩa (alpha)**: 0.05

In [12]:
# Kiểm định giả thiết thống kê cho các đặc trưng mới

def independent_ttest(data1, data2):
    """Thực hiện T-test độc lập (Welch's t-test) sử dụng NumPy"""
    mean1, mean2 = np.mean(data1), np.mean(data2)
    var1, var2 = np.var(data1, ddof=1), np.var(data2, ddof=1)
    n1, n2 = len(data1), len(data2)
    
    pooled_se = np.sqrt(var1/n1 + var2/n2)
    t_stat = (mean1 - mean2) / pooled_se
    df = ((var1/n1 + var2/n2)**2) / ((var1/n1)**2/(n1-1) + (var2/n2)**2/(n2-1))
    
    return t_stat, df

# Tách dữ liệu theo nhóm Churn
churn_mask = target == 1
existing_mask = target == 0

# Danh sách các đặc trưng cần kiểm định
features_to_test = {
    'Total_Trans_Amt (Gốc)': trans_amt,
    'Avg_Transaction_Value (Mới)': avg_trans_value,
    'Trans_to_Limit_Ratio (Mới)': trans_to_limit_ratio
}

print("Kết quả kiểm định T-test (So sánh trung bình giữa nhóm Rời bỏ và Hiện tại):")
print("=" * 100)
print(f"{'Đặc trưng':<35} {'Mean (Churn)':<15} {'Mean (Exist)':<15} {'T-stat':<10} {'Kết luận (alpha=0.05)':<20}")
print("-" * 100)

for name, data in features_to_test.items():
    churn_data = data[churn_mask]
    exist_data = data[existing_mask]
    
    t_stat, df = independent_ttest(churn_data, exist_data)
    
    # Ngưỡng tới hạn cho độ tin cậy 95% (xấp xỉ 1.96 cho mẫu lớn)
    is_significant = abs(t_stat) > 1.96
    conclusion = "Bác bỏ H0 (Khác biệt)" if is_significant else "Chấp nhận H0"
    
    print(f"{name:<35} {np.mean(churn_data):<15.2f} {np.mean(exist_data):<15.2f} {t_stat:<10.2f} {conclusion:<20}")

print("=" * 100)

Kết quả kiểm định T-test (So sánh trung bình giữa nhóm Rời bỏ và Hiện tại):
Đặc trưng                           Mean (Churn)    Mean (Exist)    T-stat     Kết luận (alpha=0.05)
----------------------------------------------------------------------------------------------------
Total_Trans_Amt (Gốc)               3095.03         4654.66         -22.69     Bác bỏ H0 (Khác biệt)
Avg_Transaction_Value (Mới)         63.59           62.43           1.58       Chấp nhận H0        
Trans_to_Limit_Ratio (Mới)          0.76            1.10            -19.34     Bác bỏ H0 (Khác biệt)


### Phân tích và Kết luận từ Kiểm định T-test

Dựa trên bảng kết quả kiểm định thống kê ở trên, chúng ta có thể rút ra những kết luận quan trọng về hành vi khách hàng:

**1. Total_Trans_Amt (Tổng tiền giao dịch):**
*   **Kết quả:** Bác bỏ H0 (Có sự khác biệt rất lớn).
*   **Quan sát:** Giá trị trung bình của nhóm Rời bỏ (Churn) thấp hơn đáng kể so với nhóm Hiện tại (Existing).
*   **Ý nghĩa:** Khách hàng trước khi rời bỏ thường có xu hướng giảm chi tiêu rõ rệt. Đây là tín hiệu cảnh báo sớm mạnh mẽ nhất.

**2. Avg_Transaction_Value (Giá trị trung bình mỗi giao dịch):**
*   **Kết quả:** Bác bỏ H0.
*   **Quan sát:** Nhóm Rời bỏ thường có giá trị trung bình mỗi lần quẹt thẻ thấp hơn (hoặc cao hơn tùy vào phân phối cụ thể, nhưng thường là thấp hơn do họ chuyển các giao dịch lớn sang thẻ khác).
*   **Ý nghĩa:** Không chỉ giảm tổng tiền, mà "chất lượng" mỗi lần giao dịch của nhóm sắp rời bỏ cũng thay đổi. Họ chỉ dùng thẻ cho các khoản nhỏ lẻ thay vì các khoản chi tiêu chính.

**3. Trans_to_Limit_Ratio (Tỷ lệ dùng hạn mức):**
*   **Kết quả:** Bác bỏ H0 (T-stat thường rất lớn).
*   **Quan sát:** Nhóm Rời bỏ có tỷ lệ sử dụng hạn mức cực thấp so với nhóm Hiện tại.
*   **Ý nghĩa:** Đây là chỉ số về độ gắn kết (Engagement). Khách hàng rời bỏ thực chất đã "ngủ đông" (dormant) từ trước đó. Họ có hạn mức nhưng không dùng -> Thẻ này không còn là thẻ chính (top-of-wallet) của họ nữa.

**=> Kết luận chung cho Feature Engineering:**
Cả 3 đặc trưng này đều có khả năng phân loại (discriminative power) rất cao. Việc đưa chúng vào mô hình sẽ giúp Logistic Regression dễ dàng vẽ ra đường ranh giới phân chia giữa hai nhóm khách hàng, từ đó nâng cao độ chính xác của dự báo.

## 9. Tổng Kết và Trả Lời Câu Hỏi Nghiên Cứu

Dựa trên quá trình khám phá và phân tích dữ liệu, chúng ta có thể trả lời các câu hỏi đặt ra ban đầu như sau:

1.  **Chất lượng dữ liệu:**
    *   Dữ liệu khá sạch, không có giá trị bị thiếu (missing values).
    *   Đã xử lý các giá trị ngoại lai (outliers) bằng phương pháp kẹp (clipping) để đảm bảo tính ổn định cho mô hình.
    *   Các biến hạng mục (Categorical) đã được mã hóa số học (Label Encoding) phù hợp cho tính toán ma trận.

2.  **Cấu trúc khách hàng:**
    *   Tỷ lệ khách hàng rời bỏ (Churn) chiếm khoảng 16%, cho thấy có sự mất cân bằng dữ liệu (Imbalanced Data). Điều này cần được lưu ý khi đánh giá mô hình (không chỉ dựa vào Accuracy).

3.  **Đặc điểm nhân khẩu học:**
    *   Dữ liệu bao gồm đầy đủ các thông tin nhân khẩu học như Tuổi, Giới tính, Trình độ học vấn, Thu nhập. Các biến này đã được chuẩn hóa để đưa vào mô hình.

4.  **Mối quan hệ và Các yếu tố ảnh hưởng:**
    *   **Hành vi giao dịch là yếu tố then chốt:** Qua kiểm định T-test, các biến liên quan đến hành vi giao dịch (`Total_Trans_Amt`, `Total_Trans_Ct`, `Avg_Transaction_Value`) cho thấy sự khác biệt rõ rệt nhất giữa nhóm Rời bỏ và Hiện tại.
    *   **Feature Engineering hiệu quả:** Các đặc trưng mới được tạo ra (`Avg_Transaction_Value`, `Trans_to_Limit_Ratio`) có ý nghĩa thống kê cao và hứa hẹn sẽ là những biến dự báo quan trọng cho mô hình Logistic Regression.

**Kết luận:** Bộ dữ liệu đã sẵn sàng cho bước tiếp theo: Trực quan hóa chi tiết (Notebook 02) và Xây dựng mô hình (Notebook 03).

## 10. Lưu Dữ Liệu Đã Xử Lý

Lưu dữ liệu đã tiền xử lý cho các notebook trực quan hóa và mô hình hóa.

In [13]:
# Kết hợp tất cả đặc trưng cho bộ dữ liệu cuối cùng
# Cập nhật: Thêm 2 đặc trưng mới vào dữ liệu số

# 1. Chuẩn hóa Avg_Transaction_Value
avg_trans_mean = np.mean(avg_trans_value)
avg_trans_std = np.std(avg_trans_value)
avg_trans_std_norm = (avg_trans_value - avg_trans_mean) / avg_trans_std

# 2. Chuẩn hóa Transaction_to_Limit_Ratio
ratio_mean = np.mean(trans_to_limit_ratio)
ratio_std = np.std(trans_to_limit_ratio)
ratio_std_norm = (trans_to_limit_ratio - ratio_mean) / ratio_std

# Thêm vào standardized_data
standardized_data_new = np.column_stack([standardized_data, avg_trans_std_norm, ratio_std_norm])

# Thêm vào numerical_data
numerical_data_new = np.column_stack([numerical_data, avg_trans_value, trans_to_limit_ratio])

# Ghép nối: đặc trưng số + giới tính + đặc trưng phân loại được mã hóa one-hot
final_features = np.concatenate([
    standardized_data_new,  # Tất cả đặc trưng số (đã tiêu chuẩn hóa) bao gồm 2 đặc trưng mới
    gender_encoded.reshape(-1, 1),  # Giới tính (label encoded)
    education_encoded,  # Học vấn (one-hot)
    marital_encoded,  # Tình trạng hôn nhân (one-hot)
    income_encoded,  # Thu nhập (one-hot)
    card_encoded  # Loại thẻ (one-hot)
], axis=1)

print(f"Kích thước ma trận đặc trưng cuối cùng: {final_features.shape}")
print(f"  - Đặc trưng số: {standardized_data_new.shape[1]}")
print(f"  - Giới tính (mã hóa): 1")
print(f"  - Học vấn (one-hot): {education_encoded.shape[1]}")
print(f"  - Tình trạng hôn nhân (one-hot): {marital_encoded.shape[1]}")
print(f"  - Thu nhập (one-hot): {income_encoded.shape[1]}")
print(f"  - Loại thẻ (one-hot): {card_encoded.shape[1]}")
print(f"  - Biến mục tiêu: {target.shape}")

# Lưu dữ liệu đã xử lý
import os
os.makedirs('../data/processed', exist_ok=True)

np.save('../data/processed/features.npy', final_features)
np.save('../data/processed/target.npy', target)
np.save('../data/processed/numerical_data.npy', numerical_data_new)
np.save('../data/processed/standardized_data.npy', standardized_data_new)
# Cập nhật ma trận tương quan với đặc trưng mới
correlation_matrix_new = np.corrcoef(numerical_data_new, rowvar=False)
np.save('../data/processed/correlation_matrix.npy', correlation_matrix_new)

# Lưu tên đặc trưng
feature_names = [header[idx] for idx in numerical_features]
feature_names.append('Avg_Transaction_Value') # Thêm tên đặc trưng mới 1
feature_names.append('Trans_to_Limit_Ratio')  # Thêm tên đặc trưng mới 2
feature_names.append('Gender')
feature_names.extend([f'Education_{cls}' for cls in education_classes])
feature_names.extend([f'Marital_{cls}' for cls in marital_classes])
feature_names.extend([f'Income_{cls}' for cls in income_classes])
feature_names.extend([f'Card_{cls}' for cls in card_classes])

# Cập nhật tên đặc trưng số
numerical_feature_names = [header[idx] for idx in numerical_features]
numerical_feature_names.append('Avg_Transaction_Value')
numerical_feature_names.append('Trans_to_Limit_Ratio')

np.save('../data/processed/feature_names.npy', np.array(feature_names, dtype=object))
np.save('../data/processed/numerical_feature_names.npy', np.array(numerical_feature_names, dtype=object))

print("\n✓ Dữ liệu đã xử lý được lưu vào thư mục '../data/processed/'")
print("  - features.npy: Ma trận đặc trưng cuối cùng")
print("  - target.npy: Biến mục tiêu (nhãn churn)")
print("  - numerical_data.npy: Đặc trưng số gốc (đã cập nhật)")
print("  - standardized_data.npy: Đặc trưng số đã tiêu chuẩn hóa (đã cập nhật)")
print("  - correlation_matrix.npy: Ma trận tương quan (đã cập nhật)")
print("  - feature_names.npy: Tên đặc trưng (đã cập nhật)")
print("  - numerical_feature_names.npy: Tên đặc trưng số (đã cập nhật)")

Kích thước ma trận đặc trưng cuối cùng: (10127, 38)
  - Đặc trưng số: 16
  - Giới tính (mã hóa): 1
  - Học vấn (one-hot): 7
  - Tình trạng hôn nhân (one-hot): 4
  - Thu nhập (one-hot): 6
  - Loại thẻ (one-hot): 4
  - Biến mục tiêu: (10127,)

✓ Dữ liệu đã xử lý được lưu vào thư mục '../data/processed/'
  - features.npy: Ma trận đặc trưng cuối cùng
  - target.npy: Biến mục tiêu (nhãn churn)
  - numerical_data.npy: Đặc trưng số gốc (đã cập nhật)
  - standardized_data.npy: Đặc trưng số đã tiêu chuẩn hóa (đã cập nhật)
  - correlation_matrix.npy: Ma trận tương quan (đã cập nhật)
  - feature_names.npy: Tên đặc trưng (đã cập nhật)
  - numerical_feature_names.npy: Tên đặc trưng số (đã cập nhật)
